In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import layers
from data import dataprep

In [2]:
tf.reset_default_graph()

In [3]:
train_dict, test_dict = dataprep.mf_train_test()

In [4]:
n_user = train_dict['user_id'].max() + 1
n_movie = train_dict['movie_id'].max() + 1
n_dim = 20
reg_param = 0.01
learning_rate = 0.01

In [5]:
# users = tf.placeholder(dtype=tf.int64, shape=[None])
# movies = tf.placeholder(dtype=tf.int64, shape=[None])
# rating = tf.placeholder(dtype=tf.float32, shape=[None])

trainBatch = tf.train.shuffle_batch(train_dict, batch_size=10000, capacity=80000, min_after_dequeue=20000, 
                       num_threads=2, enqueue_many=True)

testBatch = tf.train.shuffle_batch(test_dict, batch_size=test_dict['user_id'].shape[0], 
                                   capacity=test_dict['user_id'].shape[0], min_after_dequeue=0, enqueue_many=True)

def get_train_data(): return trainBatch["user_id"], trainBatch["movie_id"], trainBatch["rating"]
def get_test_data(): return testBatch["user_id"], testBatch["movie_id"], testBatch["rating"]
is_train = tf.placeholder(dtype=bool)
users, movies, rating = tf.cond(is_train, get_train_data, get_test_data)

In [6]:
with tf.variable_scope("embedding"):
    user_weight = tf.get_variable(name="user_w",
                    shape=[n_user,n_dim], dtype=tf.float32,
                   initializer=layers.xavier_initializer())
    movie_weight = tf.get_variable(name="movie_w",
                   shape=[n_movie,n_dim], dtype=tf.float32,
                   initializer=layers.xavier_initializer())
    user_bias = tf.get_variable(name="user_b",
                   shape=[n_user],dtype=tf.float32,
                   initializer=layers.xavier_initializer())
    movie_bias = tf.get_variable(name="movie_b",
                   shape=[n_movie], dtype=tf.float32,
                   initializer=layers.xavier_initializer())
    
    for v in tf.trainable_variables():
        tf.summary.histogram(name=v.name.replace(":0", ""), values=v)

In [7]:
with tf.name_scope("inference"):
    user_emb = tf.nn.embedding_lookup(user_weight, users)
    u_b = tf.nn.embedding_lookup(user_bias, users)
    movie_emb = tf.nn.embedding_lookup(movie_weight, movies)
    m_b = tf.nn.embedding_lookup(movie_bias, movies)

In [8]:
user_emb

<tf.Tensor 'inference/embedding_lookup:0' shape=(?, 20) dtype=float32>

In [9]:
u_b.shape

TensorShape([Dimension(None)])

In [10]:
with tf.name_scope("inference"):
    pred = tf.reduce_sum(tf.multiply(user_emb,movie_emb), 1) + u_b + m_b

In [11]:
with tf.name_scope("loss"):
    reg_loss = layers.apply_regularization(layers.l2_regularizer(scale=reg_param),
                                          weights_list=(user_weight,movie_weight))
    loss = tf.nn.l2_loss(pred - rating) + reg_loss
    tf.summary.scalar("l2loss", loss)
    train_ops = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [12]:
with tf.name_scope("loss"):
    rmse = tf.sqrt(tf.reduce_mean(tf.square(pred-rating)))

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    merged = tf.summary.merge_all()
    summaries_dir = '_summary/mf'
    train_writer = tf.summary.FileWriter(summaries_dir + "/train", sess.graph)
    test_writer = tf.summary.FileWriter(summaries_dir + "/test", sess.graph)
    
#     train_input_dict = {users: train_dict['user_id'],
#                        movies: train_dict['movie_id'],
#                        rating: train_dict['rating']}
#     test_input_dict = {users: test_dict['user_id'],
#                        movies: test_dict['movie_id'],
#                        rating: test_dict['rating']}

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for i in range(200):
#         _, loss_val = sess.run([train_ops, loss], feed_dict=train_input_dict)
        _, loss_val = sess.run([train_ops, loss], feed_dict={is_train:True})
        if i % 10 == 0:
#             rmse_train = sess.run(rmse, feed_dict=train_input_dict)
#             rmse_test = sess.run(rmse, feed_dict=test_input_dict)
            rmse_train, train_summary= sess.run([rmse, merged], feed_dict={is_train:True})
            rmse_test, test_summary= sess.run([rmse, merged], feed_dict={is_train:False})
            print("RMSE TRAIN: %.3f, RMSE TEST: %.3f" % (rmse_train, rmse_test))
            train_writer.add_summary(train_summary, i)
            test_writer.add_summary(test_summary, i)
            
    coord.request_stop()
    coord.join(threads)

RMSE TRAIN: 3.687, RMSE TEST: 3.691
RMSE TRAIN: 3.478, RMSE TEST: 3.486
RMSE TRAIN: 3.088, RMSE TEST: 3.082
RMSE TRAIN: 2.257, RMSE TEST: 2.274
RMSE TRAIN: 1.243, RMSE TEST: 1.265
RMSE TRAIN: 1.077, RMSE TEST: 1.109
RMSE TRAIN: 0.962, RMSE TEST: 1.005
RMSE TRAIN: 0.929, RMSE TEST: 0.978
RMSE TRAIN: 0.896, RMSE TEST: 0.968
RMSE TRAIN: 0.901, RMSE TEST: 0.957
RMSE TRAIN: 0.897, RMSE TEST: 0.954
RMSE TRAIN: 0.886, RMSE TEST: 0.952
RMSE TRAIN: 0.878, RMSE TEST: 0.950
RMSE TRAIN: 0.878, RMSE TEST: 0.949
RMSE TRAIN: 0.887, RMSE TEST: 0.949
RMSE TRAIN: 0.865, RMSE TEST: 0.946
RMSE TRAIN: 0.856, RMSE TEST: 0.944
RMSE TRAIN: 0.859, RMSE TEST: 0.942
RMSE TRAIN: 0.843, RMSE TEST: 0.941
RMSE TRAIN: 0.837, RMSE TEST: 0.940
